# Imports

In [1]:
!pip install opencv-python

import cv2
from google.colab import drive
import pandas as pd
import os
from PIL import Image

# Load UCF11 dataset

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
dataset_path = '/content/drive/MyDrive/DAT255_Semesterproject/archive/UCF11_updated_mpg'

dataset = []
loaded_labels = [
    "basketball",
    "biking",
    "diving",
    "golf_swing",
    "horse_riding",
    "soccer_juggling",
    "swing",
    "tennis_swing",
    "trampoline_jumping",
    "volleyball_spiking",
    "walking"
]

# Iterate through the selected labels
for label in loaded_labels:
    # Construct the path to the label's subfolder
    label_path = os.path.join(dataset_path, label)

    # Check if the label folder exists
    if os.path.exists(label_path):
        # Iterate through the video files in the label folder
        for folder in os.listdir(label_path):
          folder_path = os.path.join(label_path, folder)
          for filename in os.listdir(folder_path):
              # Check if the file is a video (you can adjust the extensions as needed)
              if filename.endswith(('.avi', '.mpg')):  # Add other video extensions if needed
                  # Construct the full path to the video file
                  video_path = os.path.join(label_path, filename)

                  # Append the video path and label to the dataset
                  dataset.append([video_path, label])
# Now you have a list of tuples, where each tuple contains:
# (video_path, label)

# Example: Print the first 5 entries in the dataset
print(dataset[:5])

[['/content/drive/MyDrive/DAT255_Semesterproject/archive/UCF11_updated_mpg/basketball/v_shooting_24_01.mpg', 'basketball'], ['/content/drive/MyDrive/DAT255_Semesterproject/archive/UCF11_updated_mpg/basketball/v_shooting_24_02.mpg', 'basketball'], ['/content/drive/MyDrive/DAT255_Semesterproject/archive/UCF11_updated_mpg/basketball/v_shooting_24_06.mpg', 'basketball'], ['/content/drive/MyDrive/DAT255_Semesterproject/archive/UCF11_updated_mpg/basketball/v_shooting_24_04.mpg', 'basketball'], ['/content/drive/MyDrive/DAT255_Semesterproject/archive/UCF11_updated_mpg/basketball/v_shooting_24_05.mpg', 'basketball']]


# Load detectron2

In [10]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 7.3 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Cloning into 'detectron2'...
remote: Enumerating objects: 15837, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 15837 (delta 30), reused 12 (delta 12), pack-reused 15771 (from 2)
Receiving objects: 100% (15837/15837), 6.40 MiB | 10.32 MiB/s, done.
Resolving deltas: 100% (11532/11532), done.


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


Ignoring dataclasses: markers 'python_version < "3.7"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.6 MB/s eta 0:00:00
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61396 sha256=c4ab03f51359b598b516e4ff8a2c987faa07fc31f4ff56f90925d5be4515d99d
  Stored in directory: /root/.cache/pip/wheels/65/71/95/3b8fde5c65c6e4a806e0867c1651dcc71a1cb2f3430e8f355f
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-

In [11]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
torch:  2.5 ; cuda:  cu124
detectron2: 0.6


In [12]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# Try Detectron2 model on one of the videos

In [15]:
# prompt: Load an image from my dataset and test the model on one frame

# Load an image from your dataset
image_path = dataset[0][0] # Taking the first video path from the dataset

# Assuming you want to extract a frame (e.g., the first frame)
video = cv2.VideoCapture(image_path)
success, image = video.read()
if success:
    # Test the model on the extracted frame
    outputs = predictor(image)
    v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])
else:
    print("Failed to load the video or extract the frame.")
video.release()


Failed to load the video or extract the frame.
